In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [21]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [22]:
# load dataset
dataframe = pd.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [23]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = utils.to_categorical(encoded_Y)

In [24]:
# define baseline model
def baseline_model():
    # create model
    model = models.Sequential()
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    # Compile model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    return model


In [28]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=250, batch_size=5, verbose=0)

In [29]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [30]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 95.33% (6.00%)


# Evaluate Smaller Network

In [31]:
# define baseline model
def baseline_model():
    # create model
    model = models.Sequential()
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    # Compile model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    return model


In [32]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [33]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [34]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 96.00% (4.42%)


# Evaluate Larger Network

In [35]:
# define baseline model
def baseline_model():
    # create model
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(4, activation='relu'))   
    model.add(layers.Dense(3, activation='softmax'))
    # Compile model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    return model


In [39]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=300, batch_size=15, verbose=0)

In [37]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [38]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 82.67% (29.84%)


# Overfitting

In [46]:
# define baseline model
def baseline_model():
    # create model
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(4, activation='relu'))   
    model.add(layers.Dense(3, activation='softmax'))
    # Compile model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    return model


In [47]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=500, batch_size=15, verbose=0)

In [48]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [49]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 91.33% (11.57%)


# Rewrite Using Keras Functional API 

In [54]:
def baseline_model():
    input = keras.Input(shape=(10,))
    x = layers.Dense(10, activation = 'relu')(x)
    x = layers.Dense(4, activation = 'relu')(x)
    output = layers.Dense(3,activation='softmax')(x)
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
    return model

In [55]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [56]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [58]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

NameError: name 'keras' is not defined

# Rewrite Using Keras Model Subclass

In [59]:
import tensorflow.keras as k
class MyModel(k.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = k.layers.Dense(10, activation='relu')
    self.dense1 = k.layers.Dense(4, activation='relu')
    self.dense2 = k.layers.Dense(3, activation='softmax')

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(X)

model = MyModel()

In [60]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])